In [31]:
import pandas as pd
from datetime import datetime
import numpy as np

In [32]:
df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [33]:
def get_date(date):
    papi = [datetime(int(i[:4]),int(i[5:7]),int(i[8:10]),int(i[11:13]),int(i[14:16]),int(i[17:19])) for i in date]
    papi = [(i - min(papi)).total_seconds() for i in papi]
    papi = [i/max(papi) for i in papi]
    return papi

In [34]:
df['instock_date'] = get_date(df['instock_date'].values)
df['stock_update_date'] = get_date(df['stock_update_date'].values)
df['uk_date1'] = get_date(df['uk_date1'].values)
df['uk_date2'] = get_date(df['uk_date2'].values)
df_test['instock_date'] = get_date(df_test['instock_date'].values)
df_test['stock_update_date'] = get_date(df_test['stock_update_date'].values)
df_test['uk_date1'] = get_date(df_test['uk_date1'].values)
df_test['uk_date2'] = get_date(df_test['uk_date2'].values)

In [35]:
df['volumes'].fillna(1, inplace=True)
df_test['volumes'].fillna(1, inplace=True)

In [68]:
po = df.iloc[:,1].values
po_t = df_test.iloc[:,1].values
print(len(po),len(po_t))
po = np.append(arr=po,values=po_t)

import csv

with open('gift_type.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["gift_type"])
    for i in range(len(po)):
            writer.writerow([po[i]])

gift_type = pd.read_csv('gift_type.csv')
k = pd.get_dummies(gift_type['gift_type'])
k = k.values
print(len(k))

20279 13519
33798


In [77]:
x_train = df.iloc[:-1000,3:-2].values
y_train = df.iloc[:-1000,-1].values
x_train_test = df.iloc[-1000:,3:-2].values
y_train_test = df.iloc[-1000:,-1].values
x_test = df_test.iloc[:,3:-1].values
ids = df_test.iloc[:,0]
volumes = df.iloc[:-1000,-2].values
volumes_t = df.iloc[-1000:,-2].values
volumes_te = df_test.iloc[:,-2].values

for i in range(len(y_train)):
    y_train[i] = y_train[i]/volumes[i]

for i in range(len(y_train_test)):
    y_train[i] = y_train[i]/volumes_t[i]

x_train = np.append(arr=x_train,values=k[:-14519],axis=1)
x_train_test = np.append(arr=x_train_test,values=k[-14519:-13519],axis=1)
x_test = np.append(arr=x_test,values=k[-13519:],axis=1)

x_train_w = []
y_train_w = []
x_train_test_w = []
y_train_test_w = []

x_train_o = []
y_train_o = []
x_train_test_o = []
y_train_test_o = []

for i in range(len(x_train)):
    if(volumes[i] == 1):
        x_train_o.append(x_train[i][:-1])
        y_train_o.append(y_train[i])
    else:
        x_train_w.append(x_train[i])
        y_train_w.append(y_train[i])

for i in range(len(x_train_test)):
    if(volumes_t[i] == 1):
        x_train_test_o.append(x_train_test[i][:-1])
        y_train_test_o.append(y_train_test[i])
    else:
        x_train_test_w.append(x_train_test[i])
        y_train_test_w.append(y_train_test[i])

x_train_w = np.array(x_train_w)
x_train_o = np.array(x_train_o)
y_train_w = np.array(y_train_w)
y_train_o = np.array(y_train_o)
x_train_test_w = np.array(x_train_test_w)
x_train_test_o = np.array(x_train_test_o)
y_train_test_w = np.array(y_train_test_w)
y_train_test_o = np.array(y_train_test_o)

from sklearn.preprocessing import StandardScaler
sc_w = StandardScaler()
x_train_w = sc_w.fit_transform(x_train_w)
x_train_test_w = sc_w.transform(x_train_test_w)

sc_o = StandardScaler()
x_train_o = sc_o.fit_transform(x_train_o)
x_train_test_o = sc_o.transform(x_train_test_o)

In [83]:
print(len(x_train_test[100]),len(x_test[2000]),len(x_train[450]))

1373 1373 1373


In [73]:
import keras
from keras.layers import Dense
from keras.models import Sequential

model_w = Sequential()
model_w.add(Dense(output_dim=100,init='uniform',activation='relu',input_dim=1373))
model_w.add(Dense(output_dim=100,init='uniform',activation='relu'))
model_w.add(Dense(output_dim=100,init='uniform',activation='relu'))
model_w.add(Dense(output_dim=1,init='uniform',activation='linear'))

model_w.compile(optimizer="adam", loss="mean_squared_error",metrics=['accuracy'])
model_w.fit(x_train_w, y_train_w, epochs=100)
model_w.evaluate(x_train_test_w,y_train_test_w)

model_o = Sequential()
model_o.add(Dense(output_dim=100,init='uniform',activation='relu',input_dim=1372))
model_o.add(Dense(output_dim=100,init='uniform',activation='relu'))
model_o.add(Dense(output_dim=100,init='uniform',activation='relu'))
model_o.add(Dense(output_dim=1,init='uniform',activation='linear'))

model_o.compile(optimizer="adam", loss="mean_squared_error",metrics=['accuracy'])
model_o.fit(x_train_o, y_train_o, epochs=100)
model_o.evaluate(x_train_test_o,y_train_test_o)

us/step - loss: 0.2146 - accuracy: 1.3704e-04
Epoch 26/100
7297/7297 [==============================] - 2s 240us/step - loss: 0.1943 - accuracy: 2.7409e-04
Epoch 27/100
7297/7297 [==============================] - 2s 219us/step - loss: 0.1996 - accuracy: 1.3704e-04
Epoch 28/100
7297/7297 [==============================] - 2s 228us/step - loss: 0.1895 - accuracy: 2.7409e-04
Epoch 29/100
7297/7297 [==============================] - 2s 238us/step - loss: 0.2003 - accuracy: 2.7409e-04
Epoch 30/100
7297/7297 [==============================] - 2s 213us/step - loss: 0.1832 - accuracy: 1.3704e-04
Epoch 31/100
7297/7297 [==============================] - 1s 175us/step - loss: 0.1955 - accuracy: 1.3704e-04
Epoch 32/100
7297/7297 [==============================] - 2s 212us/step - loss: 0.1782 - accuracy: 1.3704e-04
Epoch 33/100
7297/7297 [==============================] - 2s 254us/step - loss: 0.1906 - accuracy: 1.3704e-04
Epoch 34/100
7297/7297 [==============================] - 2s 234us/step - 

[7318.249150771625, 0.0]

In [85]:
y_pred = []
for i in range(len(x_test)):
    if volumes_te[i] == 1:
        p = x_test[i]
        p = np.array(p).reshape(-1,1)
        print(len(p))
        p = sc_o.transform(p)
        y_pred.append(model_o.predict(p))
    else:
        p = x_test[i]
        p = np.array(p).reshape(-1,1)
        print(p)
        p = sc_w.transform(p)
        p = model_w.predict(p)
        p[0] = p[0]*volumes[i]
        y_pred.append(p)
for i in range(len(y_pred)):
    y_pred[i] = y_pred[i][0]

print(y_pred[10])

[[2.40000000e+01]
 [0.00000000e+00]
 [2.30103579e-02]
 ...
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]]


ValueError: non-broadcastable output operand with shape (1373,1) doesn't match the broadcast shape (1373,1373)

In [18]:
import csv

with open('predictions5.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["gift_id", "price"])
    for i in range(len(y_pred)):
            writer.writerow([ids[i], y_pred[i][0]])

In [37]:
volumes = df.iloc[:,-2].values
print(volumes)

[1. 1. 1. ... 1. 1. 1.]
